In [1]:
from merged import *

ImportError: cannot import name 'get_config'

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

In [217]:
from dependencies import *
from image_preprocess import image_preprocessing1
from one_hot_encode import one_hot_encode

In [10]:
pip install seaborn

     |████████████████████████████████| 292 kB 2.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import keras
keras.__version__

'2.3.1'

In [218]:
# from tensorflow.keras.layers import Conv2D,MaxPooling2D
# from keras.layers import Dense,LSTM,Activation,Dropout,BatchNormalization,Input,Embedding,Reshape,Concatenate, GRU
# from keras.models import Sequential,Model

DATA_PATH = "data.json"
SAVED_MODEL_PATH = "model.h5"
EPOCHS = 40
BATCH_SIZE = 32
PATIENCE = 5
LEARNING_RATE = 0.0001

In [219]:
from tensorflow.keras.layers import Dense,LSTM,Activation,Dropout,BatchNormalization,Input,Embedding,Reshape,Concatenate, GRU, Input, ZeroPadding2D, Convolution2D, MaxPooling2D
from tensorflow.keras import Sequential, Model

In [220]:
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import regularizers

In [221]:
def load_data(data_path):
    """Loads training dataset from json file.
    :param data_path (str): Path to json file containing data
    :return X (ndarray): Inputs
    :return y (ndarray): Targets
    """
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["MFCCs"])
    y = np.array(data["labels"])
#     print("Training sets loaded!")
    return X, y

In [222]:
def prepare_dataset(data_path, test_size=0.2, validation_size=0.2):
    """Creates train, validation and test sets.
    :param data_path (str): Path to json file containing data
    :param test_size (flaot): Percentage of dataset used for testing
    :param validation_size (float): Percentage of train set used for cross-validation
    :return X_train (ndarray): Inputs for the train set
    :return y_train (ndarray): Targets for the train set
    :return X_validation (ndarray): Inputs for the validation set
    :return y_validation (ndarray): Targets for the validation set
    :return X_test (ndarray): Inputs for the test set
    :return X_test (ndarray): Targets for the test set
    """

    # load dataset
    X, y = load_data(data_path)

    # create train, validation, test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to nd array
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    print(X_train.shape)
    print(y_train.shape)

    return X_train, y_train, X_validation, y_validation, X_test, y_test,y,X

In [223]:
X_train, y_train, X_validation, y_validation, X_test, y_test,y,X = prepare_dataset(DATA_PATH)

# create network
input_shape = (X_train.shape[1], X_train.shape[2],1)
# X_train.shape

(1756, 44, 13, 1)
(1756,)


In [224]:
y.shape

(2745,)

In [225]:
class0, class1, class2, class3=[],[],[],[]

# class0=y[:161]
# class0=y[0:161]
# class0=y[0:161]
# class0=y[0:161]
class0.extend([y[i] for i in range(0,161)])
class1.extend([y[i] for i in range(161,1169)])
class2.extend([y[i] for i in range(1169,2235)])
class3.extend([y[i] for i in range(2235,2745)])

In [226]:
class0 = np.asarray(class0)
class1 = np.asarray(class1)
class2 = np.asarray(class2)
class3 = np.asarray(class3)

In [227]:
class3.shape

(510,)

In [228]:
classX0, classX1, classX2, classX3=[],[],[],[]

classX0.extend([X[i] for i in range(0,161)])
classX1.extend([X[i] for i in range(161,1169)])
classX2.extend([X[i] for i in range(1169,2235)])
classX3.extend([X[i] for i in range(2235,2745)])

In [229]:
classX0 = np.asarray(classX0)
classX1 = np.asarray(classX1)
classX2 = np.asarray(classX2)
classX3 = np.asarray(classX3)
classX0.shape

(161, 44, 13)

In [230]:
classX0.shape

(161, 44, 13)

In [231]:
train_bomb_audio, test_bomb_audio, train_bomb_label, test_bomb_label = train_test_split(classX0, class0, test_size=0.3)
train_gun_audio, test_gun_audio, train_gun_label, test_gun_label = train_test_split(classX1, class1, test_size=0.3)
train_heli_audio, test_heli_audio, train_heli_label, test_heli_label = train_test_split(classX2, class2, test_size=0.3)
train_tank_audio, test_tank_audio, train_tank_label, test_tank_label = train_test_split(classX3, class3, test_size=0.3)

In [232]:
helicopter_file_path="/home/emondey/Downloads/ARL/Dataset/helicopter"
tank_file_path="/home/emondey/Downloads/ARL/Dataset/tank"

bomb_file_path="/home/emondey/Downloads/ARL/Dataset/bomb"
gun_file_path="/home/emondey/Downloads/ARL/Dataset/gun"


image_list_helicopter=os.listdir(helicopter_file_path)
image_list_tank=os.listdir(tank_file_path)

image_list_bomb=os.listdir(bomb_file_path)
image_list_gun=os.listdir(gun_file_path)

In [233]:
train_heli_image, test_heli_image = train_test_split(image_list_helicopter, test_size=0.3)

In [234]:
train_heli_image=np.random.choice(image_list_helicopter,int(np.floor(len(image_list_helicopter)*.7)),replace=False)
test_heli_image=np.random.choice(image_list_helicopter,int(np.ceil(len(image_list_helicopter)*.3)),replace=False)
train_bomb_image=np.random.choice(image_list_bomb,int(np.floor(len(image_list_bomb)*.7)),replace=False)
test_bomb_image=np.random.choice(image_list_bomb,int(np.ceil(len(image_list_bomb)*.3)),replace=False)


train_tank_image=np.random.choice(image_list_tank,int(np.floor(len(image_list_tank)*.7)),replace=False)
test_tank_image=np.random.choice(image_list_tank,int(np.ceil(len(image_list_tank)*.3)),replace=False)
train_gun_image=np.random.choice(image_list_gun,int(np.floor(len(image_list_gun)*.7)),replace=False)
test_gun_image=np.random.choice(image_list_gun,int(np.ceil(len(image_list_gun)*.3)),replace=False)

In [236]:
# label generator

#train_label_tank=[0]*len(train_tank_image)

train_label_bomb=[0]*len(train_bomb_image)
train_label_gun=[1]*len(train_gun_image)
train_label_heli=[2]*len(train_heli_image)
train_label_tank=[3]*len(train_tank_image)

#test_label_tank=[0]*len(test_tank_image)
test_label_bomb=[0]*len(test_bomb_image)
test_label_gun=[1]*len(test_gun_image)
test_label_heli=[2]*len(test_heli_image)
test_label_tank=[3]*len(test_tank_image)

train_data_heli=zip(train_heli_image,train_label_heli)
train_data_tank=zip(train_tank_image,train_label_tank)
train_data_bomb=zip(train_bomb_image,train_label_bomb)
train_data_gun=zip(train_gun_image,train_label_gun)



test_data_heli=zip(test_heli_image,test_label_heli)
test_data_tank=zip(test_tank_image,test_label_tank)

test_data_bomb=zip(test_bomb_image,test_label_bomb)
test_data_gun=zip(test_gun_image,test_label_gun)

train_data=list(train_data_bomb)+list(train_data_gun)+list(train_data_heli)+list(train_data_tank)

test_data=list(test_data_bomb)+list(test_data_gun)+list(test_data_heli)+list(test_data_tank)


img_id_train,img_id_test,label_train,label_test=[],[],[],[]

img_id_train.append([train_data[i][0] for i in range(len(train_data))])

label_train.append([train_data[j][1] for j in range(len(train_data))])


img_id_test.append([test_data[k][0] for k in range(len(test_data))])

label_test.append([test_data[l][1] for l in range(len(test_data))])



## data frame

df_train=pd.DataFrame([img_id_train[0],label_train[0]],index=["Image_ID","Label"]).T

df_test=pd.DataFrame([img_id_test[0],label_test[0]],index=["Image_ID","Label"]).T

df_train.to_csv("/home/emondey/Downloads/ARL/Dataset/train_data.csv")

df_test.to_csv("/home/emondey/Downloads/ARL/Dataset/test_data.csv")

In [237]:
image_list_helicopter=np.array(image_list_helicopter)

In [238]:
train_gun_image.shape

(707,)

In [239]:
test_heli_audio.shape

(320, 44, 13)

In [240]:
image_list_helicopter.shape

(1066,)

In [241]:
classX2.shape

(1066, 44, 13)

In [243]:
X_train_audio=np.concatenate((train_bomb_audio, train_gun_audio,train_heli_audio,train_tank_audio), axis=0)
X_test_audio=np.concatenate((test_bomb_audio, test_gun_audio,test_heli_audio,test_tank_audio), axis=0)

y_train_audio=np.concatenate((train_bomb_label, train_gun_label,train_heli_label,train_tank_label), axis=0)
y_test_audio=np.concatenate((test_bomb_label, test_gun_label,test_heli_label,test_tank_label), axis=0)

In [244]:
X_test_audio.shape

(825, 44, 13)

In [245]:
X_train_audio = X_train_audio[..., np.newaxis]
X_test_audio = X_test_audio[..., np.newaxis]

In [246]:
X_train_audio.shape

(1920, 44, 13, 1)

In [247]:
train_gun_audio.shape

(705, 44, 13)

In [259]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, ZeroPadding2D, Convolution2D, BatchNormalization

In [260]:
# from keras.layers import Conv2D, Flatten, Dense, ZeroPadding2D, Convolution2D, BatchNormalization

In [261]:
from tensorflow.keras.layers import (
    Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, Convolution2D, ZeroPadding2D, Concatenate,
    Flatten, Activation, GlobalAveragePooling2D, GlobalMaxPooling2D, add)
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
# from keras.engine import Layer, InputSpec
from tensorflow.keras.layers import Layer, InputSpec
# from keras.engine.topology import get_source_inputs
from tensorflow.keras.utils import get_source_inputs

In [262]:
# from keras.layers import Dense,LSTM,Activation,Dropout,BatchNormalization,Input,Embedding,Reshape,Concatenate, GRU
# from keras.layers import Flatten,Conv2D,MaxPooling2D,Bidirectional,concatenate, ZeroPadding2D

In [263]:
# input_shape=Input(shape=input_shape)
input_shape=Input((44, 13, 1))
x=ZeroPadding2D((1,1))(input_shape)
x=Convolution2D(64, (3, 3))(x)
x=BatchNormalization(axis=3)(x)
x=Activation("relu")(x)

x=ZeroPadding2D((1,1))(x)
x=Convolution2D(64,(3,3))(x)
x=BatchNormalization(axis=3)(x)
x=Activation("relu")(x)
x=MaxPooling2D((2,2), strides=(2,2))(x)

x=ZeroPadding2D((1,1))(x)
x=Convolution2D(128, (3, 3))(x)
x=BatchNormalization(axis=3)(x)
x=Activation("relu")(x)

x=MaxPooling2D((2,2), strides=(2,2))(x)


x=Flatten()(x)

#     x=Dense(1024,activation="relu")(x)
x=Dense(256,activation="relu")(x)
audio=Dense(128,activation="relu")(x)


# model=Model(inputs=input_shape,outputs=audio)
print(input_shape.shape)

(None, 44, 13, 1)


In [264]:
import keras
keras.__version__

'2.3.1'

In [265]:
audio.shape

TensorShape([None, 128])

In [266]:
x_input=Input((224,224,3))
x=ZeroPadding2D((1,1))(x_input)
x=Convolution2D(64, (3, 3))(x)
#     x=BatchNormalization(axis=3)(x)
x=Activation("relu")(x)

x=ZeroPadding2D((1,1))(x)
x=Convolution2D(64,(3,3))(x)
#     x=BatchNormalization(axis=3)(x)
x=Activation("relu")(x)
x=MaxPooling2D((2,2), strides=(2,2))(x)

x=ZeroPadding2D((1,1))(x)
x=Convolution2D(128, (3, 3))(x)
#     x=BatchNormalization(axis=3)(x)
x=Activation("relu")(x)


x=MaxPooling2D((2,2), strides=(2,2))(x)


x=Flatten()(x)

# 	x=Dense(1024,activation="relu")(x)
x=Dense(256,activation="relu")(x)
img=Dense(128,activation="relu")(x)

# model_img=Model(inputs=x_input,outputs=img)
print(x_input.shape)

(None, 224, 224, 3)


In [267]:
img.shape

TensorShape([None, 128])

In [268]:
level_3 = Concatenate()([img,audio])
print(level_3.shape)

(None, 256)


In [269]:
from tensorflow.keras import regularizers

In [270]:
weight_decay = 0.00001
dropout=0.2
ADD_ON_LAYERS = True
model_in = Dense(256,
    kernel_regularizer=regularizers.l2(weight_decay),
    activation='relu')(level_3)

model_in = Dense(units=128,
    kernel_regularizer=regularizers.l2(weight_decay),
    activation=None)(model_in)
if ADD_ON_LAYERS:
    model_in = BatchNormalization()(model_in)

model_in = Activation('relu')(model_in)
model_in = Dropout(dropout)(model_in)

model_in = Dense(units=64,
    kernel_regularizer=regularizers.l2(weight_decay),
    activation=None)(model_in)
if ADD_ON_LAYERS:
    model_in = BatchNormalization()(model_in)

model_in = Activation('relu')(model_in)

main_output = Dense(4, activation='softmax')(model_in)

model = Model(inputs=[x_input,input_shape], outputs=main_output)
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 44, 13, 1)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_18 (ZeroPadding2 (None, 46, 15, 1)    0           input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 44, 13, 64)   640         zero_padding2d_18[0][0]          
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
____________________________________________________________________________________________

In [273]:
model.compile(loss='categorical_crossentropy', optimizer="SGD", metrics=["accuracy"])

In [272]:
train_image_in.shape[0][0]

TypeError: 'int' object is not subscriptable

In [274]:
history = model.fit([train_image_in, X_train_audio], train_label_in, batch_size=16, epochs=2, verbose=1)

Epoch 1/2
120/120 [==============================] - 159s 1s/step - loss: 0.8371 - accuracy: 0.6990
Epoch 2/2
120/120 [==============================] - 159s 1s/step - loss: 0.2740 - accuracy: 0.9292


In [87]:
y_true, y_pred =  test_label_in, model.predict([test_image_in, X_test_audio],verbose=1)

26/26 [==============================] - 11s 423ms/step


In [56]:
test_loss, test_acc = model.evaluate([test_image_in, X_test_audio], test_label_in)

26/26 [==============================] - 11s 400ms/step - loss: 0.1564 - accuracy: 0.9770


In [57]:
test_acc

0.9769697189331055

In [58]:
test_loss

0.15638600289821625

In [61]:
y_pred

array([[0.2946376 , 0.05367059, 0.16316955, 0.48852226],
       [0.18980119, 0.06341861, 0.30482396, 0.44195625],
       [0.41147947, 0.28212193, 0.22716413, 0.07923448],
       ...,
       [0.2080019 , 0.03713591, 0.06851204, 0.68635017],
       [0.17686944, 0.0259232 , 0.09346513, 0.7037422 ],
       [0.2179962 , 0.08451525, 0.22721839, 0.47027013]], dtype=float32)

In [62]:
test_pred=np.argmax(y_pred,axis=1)

In [63]:
test_pred

array([3, 3, 0, 3, 2, 1, 0, 3, 2, 0, 2, 2, 0, 3, 3, 0, 0, 3, 3, 3, 2, 2,
       0, 0, 1, 3, 2, 2, 3, 0, 2, 0, 2, 0, 3, 2, 0, 2, 2, 2, 0, 1, 0, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [64]:
test_pred_test=np.argmax(test_label_in,axis=1)

In [65]:
test_pred_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [250]:
df_train_data=pd.read_csv("/home/emondey/Downloads/ARL/Dataset/train_data.csv")
df_test_data=pd.read_csv("/home/emondey/Downloads/ARL/Dataset/test_data.csv")

In [251]:
def image_preprocessing1(path_1,path_2,path_3,path_4,data_frame):
    image_array=[] 
    
    for img in range(len(data_frame)):

        if list(data_frame["Label"])[img]==2:

            path=path_1
        elif list(data_frame["Label"])[img]==3:

            path=path_2
        elif list(data_frame["Label"])[img]==0:

            path=path_3
        else:
            path=path_4

        img_path_match=os.path.join(path,list(data_frame["Image_ID"])[img])
        extract_img=cv2.imread(img_path_match)
        
        img_resize=cv2.resize(extract_img,(224,224))
        print(img_resize.shape)

        image_array.append(img_resize)

    return np.array(image_array)

In [252]:
train_image_in=image_preprocessing1("/home/emondey/Downloads/ARL/Dataset/helicopter","/home/emondey/Downloads/ARL/Dataset/tank","/home/emondey/Downloads/ARL/Dataset/bomb","/home/emondey/Downloads/ARL/Dataset/gun",df_train_data)/255

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

In [253]:
test_image_in=image_preprocessing1("/home/emondey/Downloads/ARL/Dataset/helicopter","/home/emondey/Downloads/ARL/Dataset/tank","/home/emondey/Downloads/ARL/Dataset/bomb","/home/emondey/Downloads/ARL/Dataset/gun",df_test_data)/255

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

In [254]:
train_label_in=one_hot_encode(list(df_train_data["Label"]),list(df_train_data["Label"]))
test_label_in=one_hot_encode(list(df_test_data["Label"]),list(df_test_data["Label"]))
print("b")

b


In [255]:
test_image_in.shape

(825, 224, 224, 3)

In [256]:
train_label_in

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [257]:
y_train_audio

array([0, 0, 0, ..., 3, 3, 3])

In [29]:
import os
import sys
import psutil
import zipfile
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import random
np.random.seed(0)
from keras.models import Sequential,Model
# from keras.layers import Dense,LSTM,Activation,Dropout,BatchNormalization,Input,Embedding,Reshape,Concatenate, GRU
# from keras.layers import Flatten,Conv2D,MaxPooling2D,Bidirectional,concatenate
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import optimizers
# import keras_metrics as km

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from keras.utils import np_utils
from sklearn.metrics import recall_score,precision_score,f1_score,accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.metrics import make_scorer
from keras.wrappers.scikit_learn import KerasClassifier

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import roc_curve, auc

from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from keras.preprocessing import sequence

# from keras_self_attention import SeqSelfAttention


from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics

from numpy import unique
from pandas import read_csv

from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.layers import Dropout
from sklearn.preprocessing import OneHotEncoder


import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras
import tensorflow_model_optimization as tfmot

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [67]:
pip install psutil

Note: you may need to restart the kernel to use updated packages.


In [68]:
pip install -q tensorflow-model-optimization

Note: you may need to restart the kernel to use updated packages.


In [59]:
import tempfile

In [60]:
_, baseline_model_accuracy = model.evaluate(
    [test_image_in, X_test_audio], test_label_in, verbose=1)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

26/26 [==============================] - 11s 407ms/step - loss: 0.1564 - accuracy: 0.9770
Baseline test accuracy: 0.9769697189331055
Saved baseline model to: /tmp/tmpv4uv3fu4.h5


In [107]:
model.save("my_model")

In [108]:
reconstructed_model = keras.models.load_model("my_model")

In [61]:
import tensorflow_model_optimization as tfmot

In [159]:


prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 4
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_samples = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_samples / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='SGD',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# model_for_pruning.summary()

/home/emondey/anaconda3/envs/ARL/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [160]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [161]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit([train_image_in, X_train_audio], train_label_in,
                  batch_size=16, epochs=2,
                  callbacks=callbacks)

2021-09-30 16:42:52.114722: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-30 16:42:52.114740: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-30 16:42:52.114802: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2021-09-30 16:42:52.114832: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-30 16:42:52.114840: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1496] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.


Epoch 1/2
  1/120 [..............................] - ETA: 11:42 - loss: 0.1535 - accuracy: 0.9375

2021-09-30 16:42:58.784065: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-30 16:42:58.784089: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-30 16:42:58.784120: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


  2/120 [..............................] - ETA: 2:52 - loss: 0.1703 - accuracy: 0.9375 

2021-09-30 16:43:00.311377: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-30 16:43:00.312781: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1496] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-09-30 16:43:00.314611: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-09-30 16:43:00.317047: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-30 16:43:00.322221: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /tmp/tmpa1y0dezg/train/plugins/profile/2021_09_30_16_43_00
2021-09-30 16:43:00.325250: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /tmp/tmpa1y0dezg/train/plugins/profile/2021_09_30_16_43_00/emondey-OMEN-30L-Desktop-GT13-0xxx.trace.json.gz


120/120 [==============================] - 188s 2s/step - loss: 0.1724 - accuracy: 0.9612
Epoch 2/2
120/120 [==============================] - 178s 1s/step - loss: 0.2310 - accuracy: 0.9623


In [162]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   [test_image_in, X_test_audio], test_label_in, verbose=1)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

26/26 [==============================] - 13s 469ms/step - loss: 0.3588 - accuracy: 0.9139
Baseline test accuracy: 0.9769697189331055
Pruned test accuracy: 0.9139394164085388


In [163]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/tmplu3rir4h.h5


In [164]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmpd03ahdyx/assets


INFO:tensorflow:Assets written to: /tmp/tmpd03ahdyx/assets
2021-09-30 16:51:29.094242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 16:51:29.094406: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-09-30 16:51:29.094463: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-09-30 16:51:29.094572: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-30 16:51:29.094660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 16:51:29.094792: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties

Saved pruned TFLite model to: /tmp/tmpmfiz6w_4.tflite


In [165]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [166]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
# print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file1)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 385282319.00 bytes
Size of gzipped pruned Keras model: 118449380.00 bytes
Size of gzipped pruned TFlite model: 115130983.00 bytes


In [167]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)

# converter.experimental_new_converter = True

converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.optimizations.append(tf.lite.Optimize.DEFAULT)

converter.target_spec.supported_types = [tf.float16]


quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmpssbh8he3/assets


INFO:tensorflow:Assets written to: /tmp/tmpssbh8he3/assets
2021-09-30 16:52:28.892000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 16:52:28.892168: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-09-30 16:52:28.892213: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-09-30 16:52:28.892357: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-30 16:52:28.892448: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 16:52:28.892585: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties

Saved quantized and pruned TFLite model to: /tmp/tmpmn6qqhac.tflite
Size of gzipped baseline Keras model: 385282319.00 bytes
Size of gzipped pruned and quantized TFlite model: 66619336.00 bytes


In [168]:
np.save("test_image_in.npy",test_image_in)
np.save("X_test_audio.npy",X_test_audio)

np.save("test_label_in.npy",test_label_in)

In [169]:
test_label_in=np.load("test_label_in.npy")
test_label_in.shape

(825, 4)

In [170]:
def evaluate_model(interpreter):
  input_index0 = interpreter.get_input_details()[0]["index"]
  input_index1 = interpreter.get_input_details()[1]["index"]
#   input_index2 = interpreter.get_input_details()[2]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  new_prediction_digits = []
  test_image_in=np.load("test_image_in.npy")
  X_test_audio=np.load("X_test_audio.npy")
#   X_test=np.load("X_test.npy")
  test_label_in=np.load("test_label_in.npy")
  for i, test_image in enumerate(test_image_in):
    if i % 100 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))

    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    #print(test_image.shape)
    
    interpreter.set_tensor(input_index0, test_image)
    
    test_image1= X_test_audio[i]

    test_image1 = np.expand_dims(test_image1, axis=0).astype(np.float32)
    #print(test_image1.shape)
    interpreter.set_tensor(input_index1, test_image1)

    #for k, test_image2 in enumerate(X_test2):
#     test_image2 = X_test2[i]
#     test_image2 = np.expand_dims(test_image2, axis=0).astype(np.float32)
#     #print(test_image2.shape)
#     interpreter.set_tensor(input_index2, test_image2)



    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.get_tensor(output_index)
    #a=output.to_numpy()
    # print(len(output))
    
    #digit = np.argmax(output()[0])
    
    #print(output)
    
    prediction_digits.append(output)
    

  print('\n')
  print(prediction_digits)
#   print(prediction_digits.dtype)
  prediction_digits = np.array(prediction_digits)
  prediction_digits = prediction_digits.reshape(825,4)
  prediction_digits = np.argmax(prediction_digits,axis=1)
  print(prediction_digits)
  test_label_in=np.argmax(test_label_in,axis=1)
  print(test_label_in)
#   print(prediction_digits.shape)
#   for i in range(0,len(prediction_digits)):
#     for j in range
#     if prediction_digits[i]>=0.50:
#         prediction_digits[i]=1
#     else:
#         prediction_digits[i]=0
#   print(prediction_digits)
#   prediction_digits_argmax = np.argmax(prediction_digits,axis=0)
#   print(prediction_digits_argmax)
#   test_label_in_argmax = np.argmax(test_label_in,axis=0)

#   for i in range(0,len(prediction_digits)):
#         new_prediction_digits[i] = np.argmax(prediction_digits[i],axis=0)

#   print(new_prediction_digits)
#   print(confusion_matrix(y_test3, prediction_digits))
#   print(classification_report(y_test3, prediction_digits))
#   # Compare prediction results with ground truth labels to calculate accuracy.
#   prediction_digits_argmax1 = np.array(prediction_digits_argmax)
#   print(prediction_digits_argmax1)
#   test_label_in_argmax1 = np.array(test_label_in_argmax)
  accuracy = (prediction_digits == test_label_in).mean()
  return accuracy, prediction_digits

In [171]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()
test_accuracy , lol= evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 100 results so far.
Evaluated on 200 results so far.
Evaluated on 300 results so far.
Evaluated on 400 results so far.
Evaluated on 500 results so far.
Evaluated on 600 results so far.
Evaluated on 700 results so far.
Evaluated on 800 results so far.


[array([[0.12665322, 0.14826606, 0.37035504, 0.3547257 ]], dtype=float32), array([[0.05551261, 0.03737972, 0.83081615, 0.07629145]], dtype=float32), array([[0.07672123, 0.6786557 , 0.14587164, 0.09875137]], dtype=float32), array([[0.12450213, 0.19209409, 0.32500058, 0.35840315]], dtype=float32), array([[0.07620262, 0.5546706 , 0.24036238, 0.12876447]], dtype=float32), array([[0.14278083, 0.74744034, 0.0382501 , 0.07152886]], dtype=float32), array([[0.20107286, 0.3824893 , 0.21712597, 0.19931188]], dtype=float32), array([[0.23312424, 0.3198869 , 0.2296208 , 0.21736814]], dtype=float32), array([[0.07639026, 0.54151434, 0.25949574, 0.12259964]], dtype=float32), array([[0.18151955, 0.3776691 , 0.24

In [149]:
lolo_mm = np.argmax(lolo_m, axis=1)

In [150]:
lolo_mm

array([2, 2, 1, 2, 0, 0, 0, 2, 1, 0, 2, 1, 1, 2, 0, 0, 0, 2, 0, 2, 2, 2,
       2, 0, 1, 0, 2, 2, 0, 2, 2, 1, 1, 0, 0, 2, 0, 2, 2, 1, 2, 0, 2, 1,
       1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [140]:
test_label_in

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [81]:
input_index = interpreter.get_output_details()
input_index

[{'name': 'Identity',
  'index': 60,
  'shape': array([1, 4], dtype=int32),
  'shape_signature': array([-1,  4], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [80]:
interpreter.get_input_details()[0]["shape"]

array([  1, 224, 224,   3], dtype=int32)

In [83]:
prediction_digits_argmax=np.argmax(prediction_digits,axis=1)

(825, 44, 13, 1)

In [91]:
test_label_in.shape

(825, 4)

In [ ]:
test_label_in_argmax=np.argmax(test_label_in,axis=1)